In [2]:
import random as rd
import time

In [3]:
#read data from file test.txt
with open('input.txt') as f:
    n = int(f.readline().strip())   #read the number of test
    data = []
    for i in f:
        data.append(list(map(int, i.split())))    #read name task, entry time, processing time, deadline

In [4]:
#the number of tasks which doesn't meet the deadline
def fitness(data, solution):
    
    """
    Do tasks in turn from left to right
    if a task miss its deadline, increase the number of violation by 1
    """
    """
    when you do tasks from left to right,(you have a list task [1,2,3,4]
    the program will visit elements from left to right
    it means that the program will consider tasks in order 1, 2, 3, and so on)
    the program keeps a variable called t to know the time after finishing a job, 
    
    this variable helps the program to determine whether the next task can be done or not?

    for example, when you finish task 1, the current time is t, 
    if the next task has an entry time is later than t, 
    the CPU will wait until that time to do that task, 
    thus the current time after task 2 will be the entry time + processing time, 
    otherwise, if the task is already in the queue(the entry time is less than the current time), 
    you need to check if you do that task whether you can finish it before the deadline or not, 
    if you can't do it, move to the next task
    """
    
    t = 0 #current time
    violation = 0
    for i in solution:
        if data[i][1] >= t:    #if the current time is smaller than the entry time
            if data[i][1] + data[i][2] <= data[i][3]:    #if the end time is sooner than the deadline
                t = data[i][1] + data[i][2]
            else:
                violation += 1    #otherwise increase violation by 1
        else:
            if t + data[i][2] <= data[i][3]:    #if do task i and does not exceed the deadline
                t = t + data[i][2]
            else:
                violation += 1    #otherwise increase violation by 1
    return violation

#initialize population that contains random individuals
def initialize(data, num):
    population = []
    arr = list(range(len(data))) #create list of task
    for i in range(num):
        rd.shuffle(arr)    #shuffle the array
        population.append(arr.copy())    #add copy of arr to pop
    return population

#get the best individual in the population
def getBest(data, population):
    selections = [(fitness(data, i), i) for i in population]
    selections.sort(key=lambda x: x[0])    #sort the population in ascending order of the number of violations
    #the first element has the smallest number of violations
    return selections[0]

#Choose parents for mating
def getParents(data, population):
    selections = [[fitness(data, i), i] for i in population]
    arr = [i[0] for i in selections]
    total_violations = sum(arr)    #the total violations of all individuals 
    max_ = (sum(arr)/len(arr))*2    #2 times of the mean of violations of the population
    selections.sort(key=lambda x: -x[0])    #sort individuals in descending order of the number of violations
    sum_ = 0
    
    #calculate the cumulative distribution probability of the violations of each individual
    # The smaller the individual violations, the greater the probability of survival
    for i in selections:
        sum_ += max_ - i[0]
        i[0] = sum_ / total_violations
    num = len(population)*2    #get the number of parents twice the size of current population
    parents = []
    current_member = 0
    #choose parents follow the distribution of violation.
    #roulette wheel algorithm
    while current_member < num:
        r = rd.uniform(0, 1)    #random uniformly element in range (0, 1) 
        i = 0
        while i < len(selections) and selections[i][0] < r:    #find the element that has the probability larger than r
            i += 1
        parents.append(selections[i][1])
        current_member += 1
    return parents

#combination
def mate(parents):
    i = 0
    children = []
    
    #combine two consecutive parents to get 2 new children
    while i < len(parents):
        p1 = parents[i]
        p2 = parents[i+1]
        cross = len(p1)//2   #choose crossing point
        child1 = p1[:cross]    #Get the first half of p1 for child1
        child2 = []
        used = set(child1)
        
        #traverse p2, get all elements that are not in child1 for child1, otherwise push to child2
        #keep the relative positon
        for j in p2:
            if j not in used:
                child1.append(j)
            else:
                child2.append(j)
        child2 += p1[cross:]    #concatenate child2 and the second half of p1
        
        #push child1 and child2 to children
        children.append(child1)
        children.append(child2)
        i += 2
    return children

def mutation(population, rate=0.2):
    #mutate only ratex100 % of population
    n = int(len(population) * rate)
    for i in range(n):
        ele = population[rd.randint(0, len(population)-1)]    #choose a random element from the population
        i, j = rd.randint(0, len(ele)-1), rd.randint(0, len(ele)-1)    #random two positons
        i, j = min(i, j), max(i, j)    #sort i, j in ascending order

        #move element at position j to position i
        while j > i:
            ele[j], ele[j-1] = ele[j-1], ele[j]    
            j -= 1

# Survivor Selection
def selection(data, population):
    selections = [(fitness(data, i), i) for i in population]
    selections.sort(key=lambda x: x[0])    #sort individuals in ascending order of the number of violations
    
    #the population includes the current population and new children
    #To keep the size of pupulation, we take only one third of the population
    #(since the size of children twice the size of the current population)
    return [i[1] for i in selections[:len(population)//3]]



#Genetic algorithms
def GA(data, num, gen):
    population = initialize(data, num)    #initialize population
    best_violation, best_solution = getBest(data, population)
    if best_violation == 0:
        #if the violation is zero, stop
        return best_solution
    while gen > 0:
        parents = getParents(data, population)    #get parents for mating
        children = mate(parents)   #generate new children
        mutation(children)    #mutate new children
        population = selection(data, population + children)    #select the best individual for the next generation
        violation, s = getBest(data, population)
        
        #update the best individual(solution)
        if violation < best_violation:
            best_violation = violation
            best_solution = s
            #print(best_violation, gen)
        gen -= 1
        
        #if the smallest number of violations is zero, stop loop
        if best_violation == 0:
            break
    print("the number of violations: ", best_violation, gen)
    return best_solution, best_violation
        

In [5]:
#set parameter for GA
#the size of the population( variable num) must be even number
num = 500 #??
gen = 500 #??
start = time.time()    #starting time
best_solution, best_violation = GA(data, num, gen)
end = time.time()    #ending time
print("Running time in seconds")
print("Running time: ", end -start)


#save results to file
answer = []
t = 0

#this part is same as fitness function
for i in best_solution:
    if data[i][1] >= t:
        if data[i][1] + data[i][2] <= data[i][3]:
            answer.append([data[i][0], data[i][1], data[i][1] + data[i][2]])
            t = data[i][1] + data[i][2]
    else:
        if t + data[i][2] <= data[i][3]:
            answer.append([data[i][0], t, data[i][2] + t])
            t = t + data[i][2]
with open('results_GA.txt', 'w') as f:
    for i in answer:
        f.write(' '.join(map(str, i)) + '\n')

the number of violations:  152 0
Running time in seconds
Running time:  124.41209602355957


In [6]:
k = 1
results = []
num = 200
gen = 400
while k <= 15: 
    with open('test/test{}.txt'.format(k)) as f:
        n = int(f.readline().strip())
        data = []
        for i in f:
            data.append(list(map(int, i.split())))
    start = time.time()
    best_solution, best_violation = GA(data, num, gen)
    end = time.time()
    results.append([best_violation, end - start])
    num += 27
    gen += 27
    k += 1

the number of violations:  0 392
the number of violations:  1 0
the number of violations:  10 0
the number of violations:  16 0
the number of violations:  18 0
the number of violations:  23 0
the number of violations:  30 0
the number of violations:  46 0
the number of violations:  56 0
the number of violations:  58 0
the number of violations:  76 0
the number of violations:  76 0
the number of violations:  82 0
the number of violations:  90 0
the number of violations:  103 0


In [7]:
with open("r_GA.txt", 'w') as f: #for creating the plot
    for i in results:
        f.write(' '.join(map(str, i)) + '\n')